In [1]:
import akshare as ak
import pandas as pd

import sys
sys.path.append('/Users/afirez/studio/python/auto-trade/examples/py_quant/common')
sys.path.append('/Users/afirez/studio/python/auto-trade/examples/py_quant/common/common_pg.py')

print("sys.path:",sys.path)
import common_pg as common_db

print(ak.__version__)

sys.path: ['/Users/afirez/studio/python/auto-trade/examples/py_quant/datasource/akshare/test', '/usr/local/anaconda3/envs/py38/lib/python38.zip', '/usr/local/anaconda3/envs/py38/lib/python3.8', '/usr/local/anaconda3/envs/py38/lib/python3.8/lib-dynload', '', '/usr/local/anaconda3/envs/py38/lib/python3.8/site-packages', '/usr/local/anaconda3/envs/py38/lib/python3.8/site-packages/IPython/extensions', '/Users/afirez/.ipython', '/Users/afirez/.local/lib/python3.8/site-packages', '/Users/afirez/studio/python/auto-trade/examples/py_quant/common', '/Users/afirez/studio/python/auto-trade/examples/py_quant/common/common_pg.py']
pg_host : 192.168.31.110 ,pg_user : postgres ,pg_db : k_house
1.6.64


In [2]:
#stock_sse_summary_df = ak.stock_sse_summary()
#print(stock_sse_summary_df)

# 接口: stock_zh_index_spot
# 目标地址: http://vip.stock.finance.sina.com.cn/mkt/#hs_s
# 描述: 中国股票指数数据, 注意该股票指数指新浪提供的国内股票指数
# 限量: 单次返回所有指数的实时行情数据
stock_zh_index_spot_df = ak.stock_zh_index_spot()
print(stock_zh_index_spot_df)

           代码    名称         最新价     涨跌额    涨跌幅          昨收          今开  \
0    sh000001  上证指数   3269.9739  -2.027 -0.062   3272.0012   3276.1966   
1    sh000002  Ａ股指数   3426.5981  -2.047 -0.060   3428.6449   3433.0527   
2    sh000003  Ｂ股指数    303.2268  -1.491 -0.489    304.7180    304.3002   
3    sh000004  工业指数   3104.2108  -8.962 -0.288   3113.1732   3117.2991   
4    sh000005  商业指数   2595.9820  -8.261 -0.317   2604.2430   2608.4552   
..        ...   ...         ...     ...    ...         ...         ...   
521  sz399994  信息安全   1407.3670 -12.003 -0.846   1419.3700   1421.9830   
522  sz399995  基建工程   4145.5490  -3.650 -0.088   4149.1990   4154.3360   
523  sz399996  智能家居   2697.0990 -26.574 -0.976   2723.6730   2729.8990   
524  sz399997  中证白酒  15980.9170 -33.369 -0.208  16014.2860  16081.9520   
525  sz399998  中证煤炭   2308.7290 -21.537 -0.924   2330.2660   2323.7380   

             最高          最低         成交量           成交额  
0     3293.6439   3246.7313   300690066  383576011593  

In [3]:
# 0. 准备工作

dbname = "k_house"
#   0.1 创建数据库连接

#  创建数据库 股票分析数据库 的连接 
# conn, curs, engine = common_db.creat_conn_with_pg(dbname=f'{dbname}_for_test')

#  创建数据库 k_house 的连接 
conn2, curs2, engine2 = common_db.creat_conn_with_pg(dbname=dbname)

In [4]:
table_name = 'stock_zh_index_spot_test'

stock_zh_index_spot_df.to_sql(table_name,
          engine2,
          index     = False,
          if_exists = 'replace')    # 写入数据库  还有一个参数是  append，那是附加在表后
# 添加索引
# curs2.execute("""CREATE  INDEX parallel_tool_1      ON  parallel_tool_并行任务分配信息表  (交易日期); 
#                 CREATE  INDEX parallel_tool_2      ON  parallel_tool_并行任务分配信息表  (分组序号);
#                 CREATE  INDEX parallel_tool_3      ON  parallel_tool_并行任务分配信息表  (处理状态);"""
#             )
# conn2.commit()

In [5]:
df = pd.read_sql_query(f"""
    SELECT *
    FROM  {table_name}
  """,
   # params = (date,),
  con = engine2,
)

df[:-5]

,代码,名称,最新价,涨跌额,涨跌幅,昨收,今开,最高,最低,成交量,成交额
0,sh000001,上证指数,3269.9739,-2.027,-0.062,3272.0012,3276.1966,3293.6439,3246.7313,300690066,383576011593
1,sh000002,Ａ股指数,3426.5981,-2.047,-0.060,3428.6449,3433.0527,3451.3555,3402.2277,300445608,383310374985
2,sh000003,Ｂ股指数,303.2268,-1.491,-0.489,304.7180,304.3002,304.6687,301.4860,219830,142060266
3,sh000004,工业指数,3104.2108,-8.962,-0.288,3113.1732,3117.2991,3130.7006,3080.8328,195093150,296025345686
4,sh000005,商业指数,2595.9820,-8.261,-0.317,2604.2430,2608.4552,2625.5753,2577.8277,16475208,16978453145
...,...,...,...,...,...,...,...,...,...,...,...
516,sz399989,中证医疗,11324.7620,-184.132,-1.600,11508.8940,11551.7600,11603.2230,11259.0570,346372932,16510410874
517,sz399990,煤炭等权,3049.1400,-25.177,-0.819,3074.3170,3066.9250,3091.0570,3034.4030,1097136960,10042153091
518,sz399991,一带一路,1999.4980,5.682,0.285,1993.8160,1995.4890,2013.1860,1984.1640,4370484684,42902231698
519,sz399992,CSWD并购,1746.9240,2.188,0.125,1744.7360,1746.8340,1759.2550,1734.9450,1842957595,19822207189
